In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
data= pd.read_csv('auto-mpg.csv')
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [2]:
len(data[data.horsepower=='?'])

6

In [3]:
for i in range(len(data)):
    if data.horsepower[i]=='?':
        data.horsepower[i]= data.horsepower[i-1]
len(data[data.horsepower=='?'])

C:\Users\user\AppData\Local\Temp\ipykernel_21824\3914764891.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.horsepower[i]= data.horsepower[i-1]


0

In [4]:
data['horsepower'] = data['horsepower'].astype(float)

In [5]:
from sklearn import preprocessing
data= data.drop(columns='car name')
data_scaled = preprocessing.normalize(data, norm='l2')
data_scaled

array([[0.00511277, 0.00227234, 0.08720114, ..., 0.00340851, 0.019883  ,
        0.00028404],
       [0.00403885, 0.00215405, 0.09423973, ..., 0.00309645, 0.01884795,
        0.00026926],
       [0.00521025, 0.00231567, 0.09204783, ..., 0.00318404, 0.0202621 ,
        0.00028946],
       ...,
       [0.01389964, 0.00173746, 0.05863912, ..., 0.00503862, 0.03561783,
        0.00043436],
       [0.01064467, 0.00152067, 0.04562   , ..., 0.0070711 , 0.03117367,
        0.00038017],
       [0.01137482, 0.00146772, 0.04366461, ..., 0.00711843, 0.03008822,
        0.00036693]])

In [6]:
data_scaled = pd.DataFrame(data_scaled, columns = ['mpg','cylinders','displacement','horsepower','weight','acceleration',
                                                  'model year','origin'])
data_scaled.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,0.005113,0.002272,0.087201,0.036926,0.995286,0.003409,0.019883,0.000284
1,0.004039,0.002154,0.094240,0.044427,0.994364,0.003096,0.018848,0.000269
2,0.005210,0.002316,0.092048,0.043419,0.994580,0.003184,0.020262,0.000289
3,0.004637,0.002319,0.088104,0.043472,0.994936,0.003478,0.020287,0.000290
4,0.004905,0.002308,0.087137,0.040395,0.995153,0.003030,0.020197,0.000289


In [7]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data_scaled, test_size=0.33)
predictor_train= train.drop("mpg",axis=1)
predictor_test= test.drop("mpg",axis=1)
response_train= train["mpg"]
response_test= test["mpg"]

In [8]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(predictor_train, response_train)
reg.coef_

array([ 0.64936072, -0.11521621, -0.15276208, -1.6624725 , -0.09441377,
        0.53898207,  0.43478508])

The coefficient for the “year” attribute is 0.62
This means that if all the other variables remain with same values and only the year increases by 1, out output (mpg) increases by 0.62

In [9]:
from sklearn.metrics import mean_squared_error
mean_squared_error(response_test, reg.predict(predictor_test))

1.4991863226908887e-06

In [10]:
from sklearn.linear_model import RidgeCV
r_alphas = np.logspace(0.1, 1, 10)
clf_ridge = RidgeCV(alphas=r_alphas).fit(predictor_train, response_train)
mean_squared_error(response_test, clf_ridge.predict(predictor_test))

2.1104205108707176e-05

In [11]:
clf_ridge.coef_

array([ 5.28775593e-05, -1.34950084e-02, -1.42412953e-03,  6.20682976e-04,
        2.00578888e-03,  8.20345135e-03,  3.63726199e-04])

In [12]:
from sklearn.linear_model import LassoCV
lassocv = LassoCV(alphas=r_alphas).fit(predictor_train, response_train)
mean_squared_error(response_test, lassocv.predict(predictor_test))

2.3230510752362007e-05

In [13]:
lassocv.coef_

array([ 0., -0., -0.,  0.,  0.,  0.,  0.])

# Question 2

In [233]:
from sklearn.datasets import load_breast_cancer
x,y = load_breast_cancer(return_X_y=True)
x.shape

(569, 30)

In [151]:
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

kfold = model_selection.KFold(n_splits = 3)
                       

bc_params = {"base_estimator__max_depth": [3,5,10,20],
          "base_estimator__min_samples_leaf": [1, 3, 5, 10],
          'max_features': [0.5, 0.7, 1.0],
          'n_estimators': [3, 5, 10, 20],
}


model = GridSearchCV(BaggingClassifier(DecisionTreeClassifier()), bc_params, cv=3, verbose=1)
model.fit(x,y)
model.best_score_, model.best_params_

Fitting 3 folds for each of 192 candidates, totalling 576 fits


(0.9613663789102386,
 {'base_estimator__max_depth': 20,
  'base_estimator__min_samples_leaf': 3,
  'max_features': 0.7,
  'n_estimators': 5})

In [154]:
from sklearn.ensemble import AdaBoostClassifier
bc_params = {'n_estimators': [3, 5, 10, 20], 'learning_rate':[0.5,1,2]}

model1 = GridSearchCV(AdaBoostClassifier(DecisionTreeClassifier()), bc_params, cv=3, verbose=1)
model1.fit(x,y)
model1.best_score_, model1.best_params_

Fitting 3 folds for each of 12 candidates, totalling 36 fits


(0.9138494384108421, {'learning_rate': 0.5, 'n_estimators': 10})

# Question 3

In [17]:
import tensorflow as tf
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images.shape, test_images.shape

((60000, 28, 28), (10000, 28, 28))

In [18]:
train_labels

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [19]:
train_images = train_images / 255.0
test_images = test_images / 255.0
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

How many neurons does the hidden layer have in the given ANN? 128 Neurons

In [160]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)


Epoch 1/10
1875/1875 [==============================] - 4s 724us/step - loss: 0.5051 - accuracy: 0.8219
Epoch 2/10
1875/1875 [==============================] - 1s 700us/step - loss: 0.3776 - accuracy: 0.8630
Epoch 3/10
1875/1875 [==============================] - 1s 690us/step - loss: 0.3380 - accuracy: 0.8763
Epoch 4/10
1875/1875 [==============================] - 1s 691us/step - loss: 0.3157 - accuracy: 0.8837
Epoch 5/10
1875/1875 [==============================] - 1s 697us/step - loss: 0.2949 - accuracy: 0.8912
Epoch 6/10
1875/1875 [==============================] - 1s 708us/step - loss: 0.2809 - accuracy: 0.8958
Epoch 7/10
1875/1875 [==============================] - 1s 683us/step - loss: 0.2697 - accuracy: 0.8996
Epoch 8/10
1875/1875 [==============================] - 1s 689us/step - loss: 0.2568 - accuracy: 0.9042
Epoch 9/10
1875/1875 [==============================] - 1s 701us/step - loss: 0.2492 - accuracy: 0.9070
Epoch 10/10
1875/1875 [==============================] - 1s 709u

In [161]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/10
1875/1875 [==============================] - 1s 675us/step - loss: 0.5267 - accuracy: 0.8179
Epoch 2/10
1875/1875 [==============================] - 1s 665us/step - loss: 0.3955 - accuracy: 0.8600
Epoch 3/10
1875/1875 [==============================] - 1s 670us/step - loss: 0.3579 - accuracy: 0.8698
Epoch 4/10
1875/1875 [==============================] - 1s 691us/step - loss: 0.3324 - accuracy: 0.8782
Epoch 5/10
1875/1875 [==============================] - 1s 682us/step - loss: 0.3151 - accuracy: 0.8841
Epoch 6/10
1875/1875 [==============================] - 1s 692us/step - loss: 0.3037 - accuracy: 0.8886
Epoch 7/10
1875/1875 [==============================] - 1s 694us/step - loss: 0.2903 - accuracy: 0.8931
Epoch 8/10
1875/1875 [==============================] - 1s 668us/step - loss: 0.2801 - accuracy: 0.8965
Epoch 9/10
1875/1875 [==============================] - 1s 670us/step - loss: 0.2723 - accuracy: 0.8975
Epoch 10/10
1875/1875 [==============================] - 1s 674u

In [162]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/10
1875/1875 [==============================] - 2s 928us/step - loss: 0.4829 - accuracy: 0.8286
Epoch 2/10
1875/1875 [==============================] - 2s 897us/step - loss: 0.3653 - accuracy: 0.8667
Epoch 3/10
1875/1875 [==============================] - 2s 892us/step - loss: 0.3262 - accuracy: 0.8799
Epoch 4/10
1875/1875 [==============================] - 2s 924us/step - loss: 0.3026 - accuracy: 0.8878
Epoch 5/10
1875/1875 [==============================] - 2s 926us/step - loss: 0.2828 - accuracy: 0.8958
Epoch 6/10
1875/1875 [==============================] - 2s 909us/step - loss: 0.2690 - accuracy: 0.8995
Epoch 7/10
1875/1875 [==============================] - 2s 916us/step - loss: 0.2583 - accuracy: 0.9032
Epoch 8/10
1875/1875 [==============================] - 2s 903us/step - loss: 0.2474 - accuracy: 0.9072
Epoch 9/10
1875/1875 [==============================] - 2s 902us/step - loss: 0.2390 - accuracy: 0.9101
Epoch 10/10
1875/1875 [==============================] - 2s 905u

In [164]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)


Epoch 1/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.5055 - accuracy: 0.8196
Epoch 2/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3885 - accuracy: 0.8585
Epoch 3/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3545 - accuracy: 0.8695
Epoch 4/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3325 - accuracy: 0.8769
Epoch 5/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3187 - accuracy: 0.8835
Epoch 6/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3030 - accuracy: 0.8870
Epoch 7/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2923 - accuracy: 0.8922
Epoch 8/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2833 - accuracy: 0.8949: 0s - loss: 0.2825 - accuracy: 
Epoch 9/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2767 - accuracy: 0.8977
Epoch 10/10
1875/1875 [===========================

In [165]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.5524 - accuracy: 0.8020
Epoch 2/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4227 - accuracy: 0.8453
Epoch 3/10
1875/1875 [==============================] - 2s 983us/step - loss: 0.3918 - accuracy: 0.8549
Epoch 4/10
1875/1875 [==============================] - 2s 987us/step - loss: 0.3707 - accuracy: 0.8635
Epoch 5/10
1875/1875 [==============================] - 2s 991us/step - loss: 0.3585 - accuracy: 0.8673
Epoch 6/10
1875/1875 [==============================] - 2s 989us/step - loss: 0.3453 - accuracy: 0.8730
Epoch 7/10
1875/1875 [==============================] - 2s 989us/step - loss: 0.3371 - accuracy: 0.8763
Epoch 8/10
1875/1875 [==============================] - 2s 977us/step - loss: 0.3271 - accuracy: 0.8799
Epoch 9/10
1875/1875 [==============================] - 2s 985us/step - loss: 0.3226 - accuracy: 0.8812
Epoch 10/10
1875/1875 [==============================] - 2s 983us/st

In [166]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(10)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4932 - accuracy: 0.8243
Epoch 2/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3749 - accuracy: 0.8632
Epoch 3/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3426 - accuracy: 0.8747
Epoch 4/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3167 - accuracy: 0.8825
Epoch 5/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3005 - accuracy: 0.8896
Epoch 6/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2855 - accuracy: 0.8935
Epoch 7/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2741 - accuracy: 0.8975
Epoch 8/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2657 - accuracy: 0.9009
Epoch 9/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2585 - accuracy: 0.9036
Epoch 10/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.247

In [ ]:
#CNN

In [5]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3) , activation="relu", data_format="channels_last"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3) , activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(10))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Lambda, MaxPooling2D # convolution layers
from keras.layers import Dense, Dropout, Flatten # core layers
train_images = train_images.reshape(-1,28,28,1)


In [7]:
model=Sequential()
  
model.add(Conv2D(filters=32, kernel_size = (3,3), activation="relu", input_shape=(28,28,1),data_format="channels_last"))
model.add(Conv2D(filters=32, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
    
model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dense(10,activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [8]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 4608)              0         
                                                                 
 dense_4 (Dense)             (None, 256)               1179904   
                                                                 
 dense_5 (Dense)             (None, 10)                2570      
                                                      

In [9]:
from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
model.fit(train_images, train_labels, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 72s 37ms/step - loss: 0.3682 - accuracy: 0.8687
Epoch 2/10
1875/1875 [==============================] - 70s 37ms/step - loss: 0.2258 - accuracy: 0.9161
Epoch 3/10
1875/1875 [==============================] - 70s 38ms/step - loss: 0.1731 - accuracy: 0.9350
Epoch 4/10
1875/1875 [==============================] - 70s 37ms/step - loss: 0.1302 - accuracy: 0.9518
Epoch 5/10
1875/1875 [==============================] - 70s 37ms/step - loss: 0.0947 - accuracy: 0.9640
Epoch 6/10
1875/1875 [==============================] - 71s 38ms/step - loss: 0.0673 - accuracy: 0.9753
Epoch 7/10
1875/1875 [==============================] - 71s 38ms/step - loss: 0.0528 - accuracy: 0.9807
Epoch 8/10
1875/1875 [==============================] - 70s 37ms/step - loss: 0.0357 - accuracy: 0.9862
Epoch 9/10
1875/1875 [==============================] - 70s 37ms/step - loss: 0.0291 - accuracy: 0.9896
Epoch 10/10
1875/1875 [==============================] - 61s 32m

In [10]:
test_labels = to_categorical(test_labels)
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 5s - loss: 0.4254 - accuracy: 0.9202 - 5s/epoch - 17ms/step

Test accuracy: 0.920199990272522


In [15]:
train_images = train_images.reshape(-1,28,28,1)
model=Sequential()
  
model.add(Conv2D(filters=32, kernel_size = (3,3), activation="relu", input_shape=(28,28,1),data_format="channels_last"))
model.add(Conv2D(filters=32, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
    
model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(10,activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
model.fit(train_images, train_labels, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 52s 28ms/step - loss: 0.3680 - accuracy: 0.8669
Epoch 2/10
1875/1875 [==============================] - 47s 25ms/step - loss: 0.2339 - accuracy: 0.9147
Epoch 3/10
1875/1875 [==============================] - 44s 23ms/step - loss: 0.1834 - accuracy: 0.9320
Epoch 4/10
1875/1875 [==============================] - 44s 24ms/step - loss: 0.1484 - accuracy: 0.9446
Epoch 5/10
1875/1875 [==============================] - 45s 24ms/step - loss: 0.1131 - accuracy: 0.9580
Epoch 6/10
1875/1875 [==============================] - 49s 26ms/step - loss: 0.0888 - accuracy: 0.9671
Epoch 7/10
1875/1875 [==============================] - 47s 25ms/step - loss: 0.0684 - accuracy: 0.9744
Epoch 8/10
1875/1875 [==============================] - 48s 25ms/step - loss: 0.0536 - accuracy: 0.9799
Epoch 9/10
1875/1875 [==============================] - 49s 26ms/step - loss: 0.0421 - accuracy: 0.9848
Epoch 10/10
1875/1875 [==============================] - 47s 25m

In [16]:
test_labels = to_categorical(test_labels)
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 1s - loss: 0.3858 - accuracy: 0.9196 - 1s/epoch - 3ms/step

Test accuracy: 0.9196000099182129


In [20]:
train_images = train_images.reshape(-1,28,28,1)
model=Sequential()
  
model.add(Conv2D(filters=16, kernel_size = (3,3), activation="relu", input_shape=(28,28,1),data_format="channels_last"))
model.add(Conv2D(filters=16, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
    
model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(10,activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
model.fit(train_images, train_labels, epochs=10)

test_labels = to_categorical(test_labels)
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/10
1875/1875 [==============================] - 24s 12ms/step - loss: 0.4107 - accuracy: 0.85340s - loss: 0.4113 - accura
Epoch 2/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.2593 - accuracy: 0.9056
Epoch 3/10
1875/1875 [==============================] - 24s 13ms/step - loss: 0.2092 - accuracy: 0.9233
Epoch 4/10
1875/1875 [==============================] - 24s 13ms/step - loss: 0.1720 - accuracy: 0.9371
Epoch 5/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.1422 - accuracy: 0.94740s - loss: 0.1424 - accu
Epoch 6/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.1161 - accuracy: 0.9570
Epoch 7/10
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0945 - accuracy: 0.9649
Epoch 8/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0769 - accuracy: 0.9714
Epoch 9/10
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0605 - accuracy: 0.9779
Epoch 10/10
18